In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import re
import glob
import pandas as pd
from natsort import natsorted
import unicodedata

### Variabili Globali

In [4]:
root_folder = 'drive/MyDrive/BERT/'
data_folder_in = 'data/Dante/'
data_folder_out = 'data/'

# PATH
DATA_PATH_IN = os.path.abspath(os.path.join(root_folder, data_folder_in))
DATA_PATH_OUT = os.path.abspath(os.path.join(root_folder, data_folder_out))

# File utilizzati per l'addestramento
data_filename = ['fiore_it.csv',
                 'fiore_it_en.csv',
                 'fiore_en_de.csv',
                 'fiore_en_fr.csv',
                 'fiore_en_es.csv',
                 'fiore_en_nl.csv',
                 'fiore_en_sv.csv',
                 'inferno_v2.csv',
                 'inferno_fr_from_en.csv',
                 'inferno_es_from_en.csv',
                 'inferno_de_from_en.csv',
                 'inferno_nl_from_en.csv',
                 'inferno_sv_from_en.csv',
                 'inferno_en.csv',
                 'inferno_gpt.csv',
                 'inferno_gpt_en.csv',
                 'inferno_gpt_fr_from_en.csv',
                 'inferno_gpt_es_from_en.csv',
                 'inferno_gpt_de_from_en.csv',
                 'inferno_gpt_nl_from_en.csv',
                 'inferno_gpt_sv_from_en.csv',
                 'purgatorio_v2.csv',
                 'purgatorio_fr_from_en.csv',
                 'purgatorio_es_from_en.csv',
                 'purgatorio_de_from_en.csv',
                 'purgatorio_nl_from_en.csv',
                 'purgatorio_sv_from_en.csv',
                 'purgatorio_en.csv',
                 'purgatorio_gpt.csv',
                 'purgatorio_gpt_en.csv',
                 'purgatorio_gpt_fr_from_en.csv',
                 'purgatorio_gpt_es_from_en.csv',
                 'purgatorio_gpt_de_from_en.csv',
                 'purgatorio_gpt_nl_from_en.csv',
                 'purgatorio_gpt_sv_from_en.csv',
                 'paradiso_v2.csv',
                 'paradiso_fr_from_en.csv',
                 'paradiso_es_from_en.csv',
                 'paradiso_de_from_en.csv',
                 'paradiso_nl_from_en.csv',
                 'paradiso_sv_from_en.csv',
                 'paradiso_en.csv',
                 'paradiso_gpt.csv',
                 'paradiso_gpt_en.csv',
                 'paradiso_gpt_fr_from_en.csv',
                 'paradiso_gpt_es_from_en.csv',
                 'paradiso_gpt_de_from_en.csv',
                 'paradiso_gpt_nl_from_en.csv',
                 'paradiso_gpt_sv_from_en.csv'
                 ]

# Colonne
ORIGINAL_COLUMN = 'Original'
TRANSLATE_COLUMN = 'Translate'
TRANSLATE_COLUMN_GPT = 'Translate_GPT_1'

file_out = 'dataset_dantesco.csv'

In [5]:
def verifica_correttezza(df):
  original = df['Original']
  translate = df['Translate']

  print(f'Esempi nel Dataset                 : {len(df)}')
  print(f'Frase più corta in Original        : {min(original, key = len)}')
  print(f'Frase più corta in Translate       : {min(translate, key = len)}')
  print(f'Frase più lunga in Original        : {max(original, key = len)}')
  print(f'Frase più lunga in Translate       : {max(translate, key = len)}')
  print('---------------------------------------------------------------------------------------')

In [6]:
def union_df_train():
  df = pd.DataFrame(columns=[ORIGINAL_COLUMN, TRANSLATE_COLUMN])
  
  for filename in data_filename:
    filename_path = os.path.abspath(os.path.join(DATA_PATH_IN, filename))

    if "fiore" in filename:
      df_train = pd.read_csv(
        filename_path,
        usecols=[ORIGINAL_COLUMN, TRANSLATE_COLUMN, TRANSLATE_COLUMN_GPT],
        dtype={ORIGINAL_COLUMN: str, TRANSLATE_COLUMN: str, TRANSLATE_COLUMN_GPT: str}
      )

      df_train = pd.concat([df_train[[ORIGINAL_COLUMN, TRANSLATE_COLUMN]], 
                            df_train[[ORIGINAL_COLUMN, TRANSLATE_COLUMN_GPT]]. \
                            rename(columns={TRANSLATE_COLUMN_GPT: TRANSLATE_COLUMN})])
    
      df = df.append(df_train)
    
    else:
      df_train = pd.read_csv(
        filename_path,
        usecols=[ORIGINAL_COLUMN, TRANSLATE_COLUMN],
        dtype={ORIGINAL_COLUMN: str, TRANSLATE_COLUMN: str}
      )
    
      print(f'File name                          : {filename}')
      verifica_correttezza(df_train)

      df = df.append(df_train)

  return df.reset_index(drop=True)

In [7]:
# Caricamento dataset
df = union_df_train()

filename_path_out = os.path.abspath(os.path.join(DATA_PATH_OUT, file_out))

df.to_csv(filename_path_out, index=False, columns=[ORIGINAL_COLUMN, TRANSLATE_COLUMN], encoding='utf-8')

File name                          : inferno_v2.csv
Esempi nel Dataset                 : 1995
Frase più corta in Original        :  che hai?"
Frase più corta in Translate       :  Qui morì
Frase più lunga in Original        :   E già venìa su per le torbide onde un fracasso d’un suon, pien di spavento, per cui tremavano amendue le sponde,  non altrimenti fatto che d’un vento impetüoso per li avversi ardori, che fier la selva e sanz’alcun rattento  li rami schianta, abbatte e porta fori
Frase più lunga in Translate       : "Certo però poco prima, se capisco bene, che venisse Colui  che trasse dal I Cerchio gli spiriti dei patriarchi, la profonda valle fetida  tremò da ogni parte, così che pensai che l'Universo sentisse amore, per il quale alcuni credono che il mondo più volte si sia convertito in caos"
---------------------------------------------------------------------------------------
File name                          : inferno_fr_from_en.csv
Esempi nel Dataset                 : 19

In [8]:
df

,Original,Translate
0,Lo Dio d’Amor con su’ arco mi trasse,Con il suo arco il Dio dell'Amore mi trafisse
1,"Perch’i’ guardava un Fior, che m’abellia,","mentre guardavo un fiore che mi piaceva,"
2,Lo quale avea piantato Cortesia,il quale aveva piantato Cortesia
3,"Nel Giardin di Piacier, e que’ vi trasse",nel giardino del Piacere. L'amore è venuto
4,"Sì tosto, c’a me parve ch’e’ volasse.","così in fretta che mi parve volare,"
...,...,...
135179,veder voleva come si convenne l’imago al cerc...,Jag ville ta reda på var man kan sätta bilden ...
135180,ma non eran da ciò le proprie penne,Mina vingar räckte inte till en sån flygtur.
135181,se non che la mia mente fu percossa da un ful...,"Plötsligt slog en chock mig i sinnet, och till..."
135182,A l’alta fantasia qui mancò possa,Min fantasi lyckades inte hålla höjden uppe.


In [ ]:
verifica_correttezza(df)